Accessing as AbhayBisht0801

Initialized MLflow to track repo "AbhayBisht0801/Loan-Defaulter-Prediction"

Repository AbhayBisht0801/Loan-Defaulter-Prediction initialized!

In [1]:
import os
os.chdir('../')


In [2]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class MLFlowTrackingConfig:
    root_dir:Path
    test_data_path:Path
    best_model:Path
    metrics_file_name:Path
    mlflow_uri:str
    confusion_matrix:Path

In [3]:
from src.Loan_defaulter.constants import *
from src.Loan_defaulter.utils.common import read_yaml,create_directories
import mlflow.pyfunc
from mlflow import MlflowClient

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH

    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        

    def get_model_tracking_config(self)->MLFlowTrackingConfig:
        config=self.config
       
        create_directories([config.mlflow.root_dir])

        model_tracking_config=MLFlowTrackingConfig(
            root_dir=config.mlflow.root_dir,
            test_data_path=config.data_transformation.split_dir,
            best_model=config.model_training.best_model,
            metrics_file_name=config.mlflow.metrics_file_name,
            confusion_matrix=config.mlflow.confusion_matrix,
            mlflow_uri="https://dagshub.com/AbhayBisht0801/Loan-Defaulter-Prediction.mlflow"
        )
        return model_tracking_config

In [5]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score
import mlflow
import mlflow.sklearn
import numpy as np
import pickle
import joblib 
from src.Loan_defaulter.utils.common import save_json,create_confusion_matrix_plot
from urllib.parse import urlparse
import dagshub

In [6]:
class MLFLOWTracking:
    def __init__(self,config=MLFlowTrackingConfig):
        self.config = config
    def evaL_metrics(self,actual,pred):
        model_precision=precision_score(actual,pred)
        model_recall=recall_score(actual,pred)
        model_accuracy=accuracy_score(actual,pred)
        model_f1_score=accuracy_score(actual,pred)
        return {'accuracy':model_accuracy,'f1_score':model_f1_score,'precision':model_precision,'recall':model_recall}
    def log_into_mlflow(self):
        
        dagshub.init(repo_owner='AbhayBisht0801', repo_name='Loan-Defaulter-Prediction', mlflow=True)
        val_data=pickle.load(open(os.path.join(self.config.test_data_path,'val.pkl'),'rb'))
        X_val=val_data[:,:-1]
        y_val=val_data[:,-1]
        
        model=pickle.load(open(self.config.best_model,'rb'))
        with mlflow.start_run():
            pred=model.predict(X_val)
            metrics=self.evaL_metrics(y_val,pred)
            print(metrics)
            mlflow.log_params(model.get_params())
            save_json(path=Path(self.config.metrics_file_name),data=metrics)
            create_confusion_matrix_plot(self.config.confusion_matrix,pred,y_val)
            
            mlflow.log_metric('accuracy',metrics['accuracy'])
            mlflow.log_metric('f1_score',metrics['f1_score'])
            mlflow.log_metric('precision',metrics['precision'])
            mlflow.log_metric('recall',metrics['recall'])
            mlflow.log_artifact(self.config.confusion_matrix,'Confusion_Matrix')
            mlflow.sklearn.log_model(model,"model",registered_model_name="KNN")


        
        



In [7]:
try:
    config=ConfigurationManager()
    mlflow_tracking_config=config.get_model_tracking_config()
    mlflow_tracking=MLFLOWTracking(config=mlflow_tracking_config)
    mlflow_tracking.log_into_mlflow()
except Exception as e:
    raise e
    

[2025-01-02 11:22:21,582:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-01-02 11:22:21,594:INFO:common:yaml file: params.yaml loaded successfully]
[2025-01-02 11:22:21,596:INFO:common:created directory at: artifacts/mlflow]
[2025-01-02 11:22:22,621:INFO:_client:HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as AbhayBisht0801

[2025-01-02 11:22:22,627:INFO:helpers:Accessing as AbhayBisht0801]
[2025-01-02 11:22:23,180:INFO:_client:HTTP Request: GET https://dagshub.com/api/v1/repos/AbhayBisht0801/Loan-Defaulter-Prediction "HTTP/1.1 200 OK"]
[2025-01-02 11:22:23,651:INFO:_client:HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "AbhayBisht0801/Loan-Defaulter-Prediction"

[2025-01-02 11:22:23,655:INFO:helpers:Initialized MLflow to track repo "AbhayBisht0801/Loan-Defaulter-Prediction"]


Repository AbhayBisht0801/Loan-Defaulter-Prediction initialized!

[2025-01-02 11:22:23,656:INFO:helpers:Repository AbhayBisht0801/Loan-Defaulter-Prediction initialized!]


c:\Users\bisht\OneDrive\Desktop\New folder\venv\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\bisht\OneDrive\Desktop\New folder\venv\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
 

{'accuracy': 0.8970403587443946, 'f1_score': 0.8970403587443946, 'precision': 0.719661205220772, 'recall': 0.9492673992673992}
[2025-01-02 11:22:45,899:INFO:common:json file saved at: artifacts\mlflow\metrics.json]


c:\Users\bisht\OneDrive\Desktop\New folder\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'KNN' already exists. Creating a new version of this model...
2025/01/02 11:26:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: KNN, version 2
Created version '2' of model 'KNN'.


In [ ]:

client = MlflowClient()
client.transition_model_version_stage(
    name="KNN", version=3, stage="Archived"
)

<ModelVersion: creation_timestamp=1735800282069, current_stage='Archived', description='', last_updated_timestamp=1735801048762, name='KNN', run_id='9c359f7297f54ff1888595e9198db1ba', run_link='', source='models:/KNN/2', status='READY', status_message='', tags={}, user_id='', version='3'>

In [14]:
X_val=pickle.load(open('artifacts\data_transformation\split_data\\val.pkl','rb'))
X_val[:,-1]

array([1., 0., 0., ..., 0., 0., 1.])